In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from mtl_pytorch.layer_node import Conv2dNode, BN2dNode

from mtl_pytorch.trainer import Trainer
from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

from mobilenetv2 import mobilenet_v2

from mtl_model import mtl_model

from data.dataloader.nyuv2_dataloader import NYU_v2
from data.metrics.pixel2pixel_loss import NYUCriterions
from data.metrics.pixel2pixel_metrics import NYUMetrics

from data.dataloader.taskonomy_dataloader import Taskonomy
from data.metrics.pixel2pixel_loss import TaskonomyCriterions
from data.metrics.pixel2pixel_metrics import TaskonomyMetrics

: 

# Data

In [6]:
dataroot = 'datasets/Cityscapes/'

headsDict = nn.ModuleDict()
trainDataloaderDict = {}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}
for task in tasks:
    headsDict[task] = ASPPHeadNode(1280, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

# Model

In [7]:
mtlmodel = mobilenet_v2(False, heads_dict==headsDict)

# Train

In [8]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

### pre_train

In [9]:
trainer.pre_train(iters=10000, lr=0.0001, savePath=checkpoint+'Cityscapes/')

100%|██████████| 1/1 [00:44<00:00, 44.65s/it]


### alter_train

In [10]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=20000, policy_network_iters=(100,400), policy_lr=0.01, network_lr=0.0001,
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

100%|██████████| 1/1 [00:18<00:00, 18.01s/it]


### sample policy from trained policy distribution and save

In [11]:
policy_list = {'segment_semantic': [], 'depth_zbuffer': []}
name_list = {'segment_semantic': [], 'depth_zbuffer': []}

In [12]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name and not torch.eq(param, torch.tensor([0., 0., 0.])).all():
        if 'segment_semantic' in name:
            policy_list['segment_semantic'].append(param.data.cpu().detach().numpy())
            name_list['segment_semantic'].append(name)
        elif 'depth_zbuffer' in name:
            policy_list['depth_zbuffer'].append(param.data.cpu().detach().numpy())
            name_list['depth_zbuffer'].append(name)

In [13]:
from collections import OrderedDict
from scipy.special import softmax

sample_policy_dict = OrderedDict()
for task in tasks:
    for name, policy in zip(name_list[task], policy_list[task]):
        # distribution = softmax(policy, axis=-1
        distribution = softmax(policy, axis=-1)
        distribution /= sum(distribution)

        choice = np.random.choice((0,1,2), p=distribution)
        if choice == 0:
            sample_policy_dict[name] = torch.tensor([1.0,0.0,0.0])
        elif choice == 1:
            sample_policy_dict[name] = torch.tensor([0.0,1.0,0.0])
        elif choice == 2:
            sample_policy_dict[name] = torch.tensor([0.0,0.0,1.0])

In [14]:
print(sample_policy_dict)

OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 1., 0.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])


In [15]:
sample_path = 'checkpoint/CityScapes/'
# app.run(debug=True, use_reloader=False)
sample_state = {'state_dict': sample_policy_dict}
torch.save(sample_state, sample_path + 'sample_policy.model')

### post train from scratch

In [16]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=20000, lr=0.001,
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/',
                   reload='sample_policy.model')

Task Policy:
OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 1., 0.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])


### get the validation results in the paper

In [ ]:
for name, param in mtlmodel.named_parameters():
    print(name, param)
mtlmodel.load_state_dict(torch.load('CityScapes.model', map_location=torch.device('cpu')))
trainer.validate('mtl', hard=True) 

# Inference

In [7]:
x = torch.from_numpy(np.random.random([32, 3, 3, 3])).float()
# mtlmodel.load_state_dict(torch.load('CityScapes.model', map_location=torch.device('cpu')))
output = mtlmodel(x, task='segment_semantic', hard=True)

tensor([[[[0.0571, 0.2801, 0.5764],
          [0.3677, 0.7520, 0.2789],
          [0.0241, 0.0108, 0.6907]],

         [[0.2849, 0.4900, 0.5635],
          [0.5711, 0.9874, 0.8800],
          [0.3740, 0.7746, 0.2869]],

         [[0.9779, 0.9676, 0.4682],
          [0.1844, 0.2161, 0.5437],
          [0.9895, 0.3826, 0.5238]]],


        [[[0.3233, 0.3036, 0.7744],
          [0.0696, 0.3900, 0.5583],
          [0.7250, 0.3157, 0.7931]],

         [[0.5041, 0.0995, 0.7459],
          [0.8109, 0.8774, 0.0428],
          [0.8579, 0.8018, 0.1505]],

         [[0.0056, 0.9682, 0.2665],
          [0.8756, 0.2590, 0.5863],
          [0.3500, 0.6540, 0.0174]]],


        [[[0.5727, 0.9846, 0.8198],
          [0.4826, 0.5311, 0.4319],
          [0.8454, 0.6797, 0.1137]],

         [[0.4813, 0.6936, 0.8828],
          [0.4113, 0.5065, 0.1531],
          [0.8214, 0.1739, 0.6575]],

         [[0.5113, 0.8164, 0.5099],
          [0.1628, 0.2738, 0.0112],
          [0.8979, 0.9199, 0.4446]]],


    